# Paso 1: Importar Librerías de Python


In [3]:
#pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163490 sha256=2bc40e1ee6d6fc15b77eb5289a809c020ede91026b9c2e5fc217586b5c01f8dc
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [4]:
# Análisis de datos y manipulación
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Utilidades de fechas
from datetime import datetime

# Integración con Google Colab
from google.colab import drive

# Machine Learning: Preprocesamiento y división de datos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Machine Learning: Métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity

# Machine Learning: Modelado y validación
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split, GridSearchCV

# Deep Learning: Construcción de modelos y capas
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout, Dot

# Deep Learning: Optimización y regularización
from keras.optimizers import Adam
from keras.regularizers import l2

# Deep Learning: Callbacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


# Paso 2: Lectura del Dataset

In [6]:
df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')  # Descomenta y proporciona la ruta si estás cargando un archivo CSV

<ipython-input-6-6d023f7e94e1>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')  # Descomenta y proporciona la ruta si estás cargando un archivo CSV


In [7]:
# Eliminemos posibles duplicados:
df = df.drop_duplicates()

In [8]:
df.shape

(4641903, 10)

# Modelo de Filtrado Colaborativo usando Embedding con Keras:

Funcionamiento:

Los embeddings son representaciones vectoriales densas y de baja dimensión de ítems y/o usuarios. Estas representaciones capturan relaciones semánticas entre ítems o entre usuarios.

Un método popular para generar embeddings es la factorización de matrices, como la descomposición en valores singulares (SVD). En el contexto de sistemas de recomendación, se busca factorizar la matriz de interacciones usuario-ítem en dos matrices más pequeñas (una para los usuarios y otra para los ítems) cuyo producto aproximado reproduce la matriz original lo mejor posible.

Una vez que se han obtenido los embeddings, la predicción de una calificación o interacción entre un usuario e ítem se realiza tomando el producto escalar entre sus embeddings respectivos.

Ventajas:

Simplicidad y eficiencia en términos computacionales.

Puede manejar grandes conjuntos de datos debido a su naturaleza de baja dimensión.

Es efectivo para capturar patrones subyacentes en los datos.

Desventajas:

No tiene en cuenta características adicionales de usuarios o ítems.

Dificultades para manejar nuevos ítems o usuarios (problema de arranque en frío).

El código ahora incluye regularización en los embeddings, un callback para detener el entrenamiento si no hay mejoras (early stopping) y otro para reducir la tasa de aprendizaje si el error de validación no mejora (reduce learning rate on plateau).

In [9]:
data = df

In [10]:
# Crear un LabelEncoder para cada columna
reviewerID_encoder = LabelEncoder()
asin_encoder = LabelEncoder()

# Ajustar y transformar las columnas
data['reviewerID_encoded'] = reviewerID_encoder.fit_transform(data['reviewerID'])
data['asin_encoded'] = asin_encoder.fit_transform(data['asin'])

# Número de usuarios únicos y número de ítems únicos
n_users = len(np.unique(data['reviewerID_encoded']))
n_items = len(np.unique(data['asin_encoded']))

# Dimensiones del embedding
embedding_dim = 10

# Entradas
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

# Embeddings con regularización
user_embedding = Embedding(n_users, embedding_dim, embeddings_regularizer=l2(1e-6))(user_input)
item_embedding = Embedding(n_items, embedding_dim, embeddings_regularizer=l2(1e-6))(item_input)

# Producto punto para predecir la valoración/rating
merged = Dot(axes=2)([user_embedding, item_embedding])
merged = Flatten()(merged)

# Modelo
model = Model(inputs=[user_input, item_input], outputs=merged)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

verbose=0: No muestra ninguna barra de progreso ni métricas.

verbose=1: Muestra una barra de progreso y actualiza las métricas después de cada lote.

verbose=2: Muestra las métricas después de cada época, pero no muestra la barra de progreso.

In [11]:
from sklearn.model_selection import train_test_split

# Crear los arrays con los datos
user_ids = data['reviewerID_encoded'].values
item_ids = data['asin_encoded'].values
ratings = data['overall'].values

# Dividir en train y test
(user_ids_train, user_ids_test, item_ids_train, item_ids_test, ratings_train, ratings_test) = train_test_split(
    user_ids, item_ids, ratings,test_size=0.3,random_state=42)

In [12]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [13]:
%%time
# Entrenar el modelo con el conjunto de entrenamiento
model.fit([user_ids_train, item_ids_train], ratings_train, validation_data=([user_ids_test, item_ids_test], ratings_test), epochs=500, batch_size=40000, verbose=1, callbacks=[early_stop, reduce_lr])

predictions = model.predict([user_ids_test, item_ids_test])

Epoch 1/500
82/82 [==============================] - 6s 12ms/step - loss: 19.5093 - mae: 4.2587 - val_loss: 19.5115 - val_mae: 4.2592 - lr: 0.0010
Epoch 2/500
82/82 [==============================] - 1s 10ms/step - loss: 19.4934 - mae: 4.2572 - val_loss: 19.5033 - val_mae: 4.2584 - lr: 0.0010
Epoch 3/500
82/82 [==============================] - 1s 10ms/step - loss: 19.4565 - mae: 4.2529 - val_loss: 19.4787 - val_mae: 4.2552 - lr: 0.0010
Epoch 4/500
82/82 [==============================] - 1s 10ms/step - loss: 19.3695 - mae: 4.2424 - val_loss: 19.4318 - val_mae: 4.2488 - lr: 0.0010
Epoch 5/500
82/82 [==============================] - 1s 10ms/step - loss: 19.2084 - mae: 4.2225 - val_loss: 19.3651 - val_mae: 4.2389 - lr: 0.0010
Epoch 6/500
82/82 [==============================] - 1s 10ms/step - loss: 18.9613 - mae: 4.1911 - val_loss: 19.2845 - val_mae: 4.2260 - lr: 0.0010
Epoch 7/500
82/82 [==============================] - 1s 10ms/step - loss: 18.6282 - mae: 4.1477 - val_loss: 19.1961 - 

In [14]:
# Evaluar el modelo con el conjunto de prueba
loss, mae = model.evaluate([user_ids_test, item_ids_test], ratings_test, batch_size=40000)
print(f'Test Loss: {loss:.4f}')
print(f'Test MAE: {mae:.4f}')

35/35 [==============================] - 0s 3ms/step - loss: 9.2241 - mae: 1.9862
Test Loss: 9.2241
Test MAE: 1.9862


In [15]:
# MSE
mse = mean_squared_error(ratings_test, predictions)
print(f"MSE: {mse:.4f}")

# RMSE
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

# MAE
mae = mean_absolute_error(ratings_test, predictions)
print(f"MAE: {mae:.4f}")

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((ratings_test - predictions.flatten()) / ratings_test)) * 100
print(f"MAPE: {mape:.2f}%")

MSE: 6.4101
RMSE: 2.5318
MAE: 1.9862
MAPE: 50.87%


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa con Keras:

Funcionamiento:

Estos sistemas toman características de los ítems y/o usuarios y las pasan a través de una o varias capas de neuronas para obtener una predicción.
Las redes neuronales son capaces de capturar interacciones no lineales entre características, lo que las hace poderosas para tareas de modelado complejas.

Ventajas:

Capacidad de modelar relaciones no lineales.

Flexibilidad para incorporar múltiples fuentes de datos o características.

Puede manejar arranques en frío al incorporar características de nuevos ítems o usuarios.

Desventajas:

Mayor costo computacional en comparación con los sistemas basados únicamente en embeddings.

Riesgo de sobreajuste si no se tiene un conjunto de datos lo suficientemente grande.

Ahora el código incluye regularización en los embeddings, early stopping para detener el entrenamiento si el modelo deja de mejorar en el conjunto de validación, y reducción de la tasa de aprendizaje si el error en el conjunto de validación no mejora después de algunas épocas. Estas adiciones deberían ayudar a mejorar la capacidad de generalización del modelo.

In [16]:
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
hidden_units = [128, 64, 32]
dropout_rate = 0.2
l2_reg = 1e-6

# Arquitectura del modelo
user_input = Input(shape=[1], name='user_input')
item_input = Input(shape=[1], name='item_input')
concat = Concatenate()([user_input, item_input])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu', kernel_regularizer=l2(l2_reg))(dense)
    dense = Dropout(dropout_rate)(dense)
output = Dense(1, kernel_regularizer=l2(l2_reg))(dense)  # También puedes aplicar regularización L2 aquí
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

In [17]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [18]:
# Datos para entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# Dividir los datos
user_data_train, user_data_val, item_data_train, item_data_val, ratings_train, ratings_val = train_test_split(
    user_data, item_data, rating_data, test_size=0.3, random_state=42)

In [19]:
%%time
# Entrenamiento del modelo
history = model.fit([user_data_train, item_data_train], ratings_train,
    epochs=500,validation_data=([user_data_val, item_data_val], ratings_val),
    batch_size=40000,verbose=1,callbacks=[early_stop, reduce_lr])

Epoch 1/500
82/82 [==============================] - 3s 10ms/step - loss: 375362464.0000 - val_loss: 5601.9517 - lr: 0.0010
Epoch 2/500
82/82 [==============================] - 1s 8ms/step - loss: 11112158.0000 - val_loss: 19.8341 - lr: 0.0010
Epoch 3/500
82/82 [==============================] - 1s 8ms/step - loss: 2988284.7500 - val_loss: 20.3201 - lr: 0.0010
Epoch 4/500
82/82 [==============================] - 1s 8ms/step - loss: 1678561.2500 - val_loss: 19.5972 - lr: 0.0010
Epoch 5/500
82/82 [==============================] - 1s 8ms/step - loss: 1149917.3750 - val_loss: 19.6074 - lr: 0.0010
Epoch 6/500
82/82 [==============================] - 1s 8ms/step - loss: 900163.6250 - val_loss: 19.6201 - lr: 0.0010
Epoch 7/500
82/82 [==============================] - 1s 8ms/step - loss: 731520.1250 - val_loss: 19.6298 - lr: 0.0010
Epoch 8/500
82/82 [==============================] - 1s 8ms/step - loss: 596954.8750 - val_loss: 19.6109 - lr: 0.0010
Epoch 9/500
82/82 [==========================

In [20]:
# Evaluar el modelo
y_true = ratings_val
predictions = model.predict([user_data_val, item_data_val])

43518/43518 [==============================] - 69s 2ms/step


In [21]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# Funciones MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 1.3605828830348343
RMSE: 1.166440261237083
MAE: 0.9251185466427577
MAPE: 38.36791751953778%


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa y Embedding con Keras:

Otra enfoque popular para filtrado colaborativo es utilizar redes neuronales multicapa, que esencialmente aprenden características no lineales de los datos. Vamos a construir un modelo que fusiona los embeddings de usuarios y artículos (por ejemplo, películas) en una red neuronal densa.

Este enfoque, que combina embeddings y redes neuronales multicapa, puede capturar interacciones más complejas y no lineales entre usuarios y artículos. Sin embargo, es crucial prestar atención al sobreajuste y asegurarse de que el modelo no esté simplemente memorizando los datos. Por lo tanto, es recomendable emplear técnicas de regularización, ajustar hiperparámetros y validar el rendimiento con un conjunto de datos de validación.

Funcionamiento:

Combina lo mejor de ambos mundos. Primero, se utilizan embeddings para convertir ítems y usuarios en representaciones vectoriales densas. Luego, estas representaciones se pasan a través de una red neuronal para hacer la predicción.

El proceso generalmente comienza con capas de embedding que convierten identificadores de usuarios e ítems en vectores. Estos vectores luego se pasan a través de capas densas para obtener la predicción final.

Ventajas:

Capacidad de capturar patrones subyacentes en los datos mediante embeddings y modelar interacciones no lineales mediante la red neuronal.

Flexibilidad para incorporar características adicionales.

Potencialmente más preciso que cualquiera de los otros dos métodos por separado.

Desventajas:

Mayor complejidad en el modelado y entrenamiento.

Mayor costo computacional.

Requiere un ajuste más cuidadoso y riesgo de sobreajuste si no se gestiona adecuadamente.

Regularización L2: Agregar una regularización L2 a las capas de embedding y densas para prevenir el sobreajuste.

Early Stopping: Añadir una callback de early stopping para detener el entrenamiento cuando no haya mejora en el conjunto de validación durante un cierto número de épocas.

Ajuste de la tasa de aprendizaje: Utilizar la callback ReduceLROnPlateau para reducir la tasa de aprendizaje cuando no haya mejora en el conjunto de validación.

Métricas adicionales: Puedes monitorizar otras métricas como el error absoluto medio (MAE) durante el entrenamiento.

In [22]:
# Convertir reviewerID y asin a índices numéricos secuenciales
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
n_latent_factors = 50
hidden_units = [128, 64]
dropout_rate = 0.2
l2_reg = 1e-4

# Arquitectura del modelo con regularización L2
user_embedding = Embedding(n_users, n_latent_factors, embeddings_regularizer=l2(l2_reg), name='user_embedding')(user_input)
item_embedding = Embedding(n_items, n_latent_factors, embeddings_regularizer=l2(l2_reg), name='item_embedding')(item_input)

user_vec = Flatten(name='flatten_users')(user_embedding)
item_vec = Flatten(name='flatten_items')(item_embedding)
concat = Concatenate()([user_vec, item_vec])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu', kernel_regularizer=l2(l2_reg))(dense)
    dense = Dropout(dropout_rate)(dense)
output = Dense(1, kernel_regularizer=l2(l2_reg))(dense)
model = Model(inputs=[user_input, item_input], outputs=output)

model.compile(optimizer=Adam(0.001), loss='mean_squared_error', metrics=['mae'])

In [23]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [24]:
# Entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# Usando validation_split, divide los datos
train_size = int(0.7 * len(user_data))
user_data_train, user_data_val = user_data[:train_size], user_data[train_size:]
item_data_train, item_data_val = item_data[:train_size], item_data[train_size:]
y_true = rating_data[train_size:]

In [25]:
%%time
history = model.fit([user_data_train, item_data_train], rating_data[:train_size],
                    epochs=500,validation_data=([user_data_val, item_data_val], y_true),
                    batch_size=40000,verbose=1,callbacks=[early_stop, reduce_lr])

Epoch 1/500
82/82 [==============================] - 6s 49ms/step - loss: 10.2316 - mae: 2.4053 - val_loss: 5.1458 - val_mae: 2.0052 - lr: 0.0010
Epoch 2/500
82/82 [==============================] - 3s 42ms/step - loss: 1.8189 - mae: 0.9057 - val_loss: 3.9887 - val_mae: 1.7391 - lr: 0.0010
Epoch 3/500
82/82 [==============================] - 3s 42ms/step - loss: 1.6130 - mae: 0.8550 - val_loss: 3.1395 - val_mae: 1.5108 - lr: 0.0010
Epoch 4/500
82/82 [==============================] - 3s 42ms/step - loss: 1.4873 - mae: 0.8270 - val_loss: 2.4850 - val_mae: 1.3118 - lr: 0.0010
Epoch 5/500
82/82 [==============================] - 3s 42ms/step - loss: 1.3855 - mae: 0.8021 - val_loss: 2.0791 - val_mae: 1.1533 - lr: 0.0010
Epoch 6/500
82/82 [==============================] - 3s 43ms/step - loss: 1.3243 - mae: 0.7847 - val_loss: 1.9432 - val_mae: 1.0651 - lr: 0.0010
Epoch 7/500
82/82 [==============================] - 3s 42ms/step - loss: 1.2835 - mae: 0.7705 - val_loss: 1.8058 - val_mae: 0.99

In [26]:
# Predicciones
predictions = model.predict([user_data_val, item_data_val])

43518/43518 [==============================] - 68s 2ms/step


In [27]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 1.476765427193881
RMSE: 1.2152223776716264
MAE: 0.8742751385670862
MAPE: 40.65795777215035%


Estas métricas te ayudarán a tener una mejor idea del desempeño de tu modelo. Por ejemplo:

MSE y RMSE son útiles cuando quieres penalizar grandes errores.

MAE te da una idea del error medio sin considerar la dirección del error.

MAPE es útil cuando quieres representar el error en términos porcentuales.

Para una evaluación completa, es recomendable utilizar un conjunto de validación aparte (es decir, no solo depender del validation_split). Esto asegura que estás evaluando el desempeño en datos que el modelo nunca ha visto durante el entrenamiento.

En resumen, la elección del tipo de sistema de recomendación dependerá de la naturaleza del conjunto de datos, las características disponibles, las capacidades computacionales y el tipo de relaciones o interacciones que se desean capturar. A menudo, un enfoque híbrido (como el sistema que combina embeddings y redes neuronales) ofrece un buen equilibrio entre precisión y eficiencia.